### Import Libraries

In [1]:
import os
from textwrap import dedent
from dotenv import load_dotenv
import pandas as pd
from typing_extensions import TypedDict

from langgraph.graph import StateGraph
from langchain_openai import AzureChatOpenAI
from langchain_ollama import ChatOllama
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Load environment variables
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), verbose=True, override=True)

# Load environment variables
load_dotenv()

os.environ["AZURE_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_ENDPOINT"] = os.getenv("AZURE_ENDPOINT")
os.environ["AZURE_API_VERSION"] = os.getenv("AZURE_OPENAI_API_VERSION") 
os.environ["GEMINI_API_KEY"] = os.getenv("GEMINI_API_KEY")

### Generate Markdown file for Fund Flow

In [2]:
def extract_text_from_excel(file, sheetnames):
    if len(sheetnames)>0:
        for sheet in sheetnames:
            df = pd.read_excel(file, engine='openpyxl', sheet_name=sheet)
            df_cleaned = df.dropna(how='all')
            df2 = df_cleaned.fillna('').reset_index(drop=True)
            with open("../data/processed_inputs/fundflow.md", "a") as f:
                f.write(f"""##### {sheet} \n{str(df2.to_markdown())}\n""")
                # f.write(str(df2.to_markdown()))
    else:
        print("SheetNames not provided")

In [3]:
file_path = '../data/1. CMA_Data.xlsx'
extract_text_from_excel(file_path,sheetnames=['Fund Flow','Fund Flow2'])

### Define the LangGraph Agent for FundFlow Analyzer

In [4]:
# Define the AgentState class for BalanceSheet Analyzer
class AgentState(TypedDict):
    question: str
    fundflow_data: str
    result: str

def fundflow_analyzer_agent(state: AgentState):
    """
    Financial Analyzer Agent takes input questions and responds to the user's questions by processing the data provided.

    Returns:
    dict: The response from the LLM.
    """
    # Input User Question
    question = state["question"]

    with open("../data/processed_inputs/fundflow.md", "r") as f:
        state['fundflow_data'] = f.read()


    fundflow_data = state['fundflow_data']

    if fundflow_data is None:
        print("Failed to extract text from the image.")
        return {"error": "Failed to extract text from the image."}
    

    system = dedent("""
        You are a financial analyst reviewing a company's fund flow data.
        Analyze the provided Fund Flow Statement to assess financial health. Focus on:
        Net Cash Flow: Positive/Negative? Implications?
        Operating Cash Flow: Income conversion to cash?
        Investing Cash Flow: Capital expenditures/asset sales?
        Financing Cash Flow: Debt/Equity/Dividends?
        Liquidity: Cash & Equivalents - Short-term obligations?
        Working Capital: Changes in A/R, Inventory, A/P - Impact?
        Free Cash Flow: Cash generation after capital expenditures?
        Debt Service: Ability to meet obligations?
        Cash Flow from Operations vs. Net Income: Discrepancies?
        For each, explain its importance and implications for decision-making.
        """)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", f"Fund Flow Data: {fundflow_data}"),
            ("human", f"Question: {question}"),
        ]
    )

    # Initialize the LLM
    llm = AzureChatOpenAI(
        model="gpt-4o-mini",
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        azure_endpoint=os.getenv("AZURE_ENDPOINT"),
        api_version=os.getenv("AZURE_API_VERSION"),
        temperature=0
    )

    balancesheet_analyst = prompt | llm | StrOutputParser()

    result = balancesheet_analyst.invoke({"question": question, "fundflow_data": fundflow_data})
    state["result"] = result
    return state

workflow = StateGraph(AgentState)

workflow.add_node("fundflow_analyzer_node", fundflow_analyzer_agent)

workflow.set_entry_point("fundflow_analyzer_node")

app = workflow.compile()


### User Questions

In [5]:
user_question_1 = "Determine whether the company has a positive or negative net cash flow and the implications of this for the business."
result = app.invoke({"question": user_question_1})
print(result["result"])

To determine the company's net cash flow, we need to analyze the "Total Funds Available" and "Total Funds Used" from the Fund Flow Statement.

From the provided data:

- **Total Funds Available** (Row 23 of Fund Flow 1): 
  - For the year ended 31.03.2023: Rs. 248.81 lacs
  - For the year ending 31.03.2024: Rs. 254.47 lacs
  - For the year ending 31.03.2025: Rs. 272.58 lacs
  - For the year ending 31.03.2026: Rs. 291.37 lacs
  - For the year ending 31.03.2027: Rs. 314.77 lacs
  - For the year ending 31.03.2028: Rs. 337.14 lacs
  - For the year ending 31.03.2029: Rs. 358.03 lacs

- **Total Funds Used** (Row 23 of Fund Flow 2):
  - For the year ended 31.03.2023: Rs. 256.81 lacs
  - For the year ending 31.03.2024: Rs. 262.47 lacs
  - For the year ending 31.03.2025: Rs. 280.58 lacs
  - For the year ending 31.03.2026: Rs. 299.37 lacs
  - For the year ending 31.03.2027: Rs. 318.10 lacs
  - For the year ending 31.03.2028: Rs. 337.14 lacs
  - For the year ending 31.03.2029: Rs. 358.03 lacs

##

In [24]:
user_question_2 = "How much cash generated or used by the company's core operating activities?"
result = app.invoke({"question": user_question_2})
print(result["result"])

To assess how much cash was generated or used by the company's core operating activities, we need to focus on the "Operating Cash Flow" section of the Fund Flow Statement. This typically includes the profit before tax, adjustments for non-cash items (like depreciation), and changes in working capital (like accounts receivable, inventory, and accounts payable).

From the provided data, we can see the following relevant figures:

1. **Profit Before Tax**: 
   - For the year ended 31.03.2023: Rs. 95.16 lacs

2. **Add: Depreciation**: 
   - For the year ended 31.03.2023: Rs. 162.23 lacs

3. **Less: Taxes Paid/Payable**: 
   - For the year ended 31.03.2023: Rs. 9.22 lacs

4. **Net Funds Generated**: 
   - For the year ended 31.03.2023: Rs. 248.17 lacs

### Calculation of Operating Cash Flow:
Operating Cash Flow can be calculated as follows:

\[
\text{Operating Cash Flow} = \text{Profit Before Tax} + \text{Depreciation} - \text{Taxes Paid}
\]

Substituting the values:

\[
\text{Operating Cas

In [25]:
user_question_3 = "What is the net fund flow for the between 2022 - 2025?"
result = app.invoke({"question": user_question_3})
print(result["result"])

To calculate the net fund flow for the years 2022 to 2025, we need to look at the "SURPLUS (+) / SHORTFALL (-)" row from the Fund Flow Statement for those specific years.

From the provided data, the surplus/shortfall for the years 2022 to 2025 is as follows:

- **2022**: 113.88 lacs
- **2023**: 119.76 lacs
- **2024**: 138.11 lacs
- **2025**: 156.87 lacs

Now, we can sum these values to find the total net fund flow from 2022 to 2025:

\[
\text{Net Fund Flow} = 113.88 + 119.76 + 138.11 + 156.87
\]

Calculating this gives:

\[
\text{Net Fund Flow} = 628.62 \text{ lacs}
\]

Thus, the net fund flow for the years 2022 to 2025 is **628.62 lacs**.


In [26]:
user_question_4 = "What is the net fund flow for the year 2020?"
result = app.invoke({"question": user_question_4})
print(result["result"])

The provided fund flow data does not include a specific year labeled as "2020." The data appears to cover the years ending on March 31st from 2023 to 2029. Therefore, it is not possible to determine the net fund flow for the year 2020 based on the information given. 

If you have data for the year 2020 or if you meant a different year, please provide that information for further analysis.


In [27]:
user_question_5 = "Is there a surplus or deficit in the fund flow?"
result = app.invoke({"question": user_question_5})
print(result["result"])

To determine whether there is a surplus or deficit in the fund flow, we can analyze the "SURPLUS (+) / SHORTFALL (-)" rows from the provided data.

### Analysis of Surplus/Shortfall:

1. **Long Term Sources vs. Long Term Uses**:
   - Long Term Sources: 
     - 31.03.2023: 248.17 lacs
     - 2024: 254.05 lacs
     - 2025: 272.40 lacs
     - 2026: 291.16 lacs
     - 2027: 314.57 lacs
     - 2028: 337.14 lacs
     - 2029: 358.03 lacs
   - Long Term Uses:
     - 31.03.2023: 134.29 lacs
     - 2024: 134.29 lacs
     - 2025: 134.29 lacs
     - 2026: 134.29 lacs
     - 2027: 134.29 lacs
     - 2028: 123.07 lacs
     - 2029: 0 lacs

   - **Surplus Calculation**:
     - 31.03.2023: 248.17 - 134.29 = 113.88 lacs (Surplus)
     - 2024: 254.05 - 134.29 = 119.76 lacs (Surplus)
     - 2025: 272.40 - 134.29 = 138.11 lacs (Surplus)
     - 2026: 291.16 - 134.29 = 156.87 lacs (Surplus)
     - 2027: 314.57 - 134.29 = 180.28 lacs (Surplus)
     - 2028: 337.14 - 123.07 = 214.07 lacs (Surplus)
     - 2029: 

In [28]:
user_question_6 = "How much was spent on capital expenditures (CapEx)?"
result = app.invoke({"question": user_question_6})
print(result["result"])

Based on the provided Fund Flow Statement, there is no explicit entry for capital expenditures (CapEx) listed in the "USES" section. The line for "Increase in Fixed Assets" shows a value of 0 for all periods, indicating that there were no capital expenditures during the reported years.

Therefore, the amount spent on capital expenditures (CapEx) is **Rs. 0 lacs** for the periods shown in the statement.
